# FIE for Dirichlet BVP in a circle with Polynomial Gauss rule

In [1]:
import numpy as np
import scipy.integrate as integrate
#import scipy.interpolate as interp
%matplotlib inline
import matplotlib.pyplot as plt
import scipy
import math
import time


In [2]:
def K_matrix(phis, taus):
    """Returns the Kernel for BIE as bivariate function
    """
    return np.ones((len(phis), len(taus))) * 1/(2*np.pi)

#np.cos(theta-phi)
def ke(r, phi, theta):
    """Returns the kernel function to find 
    the solution of BIE for the hole space -
    three-variate function"""
    kernel = (1 - r*np.cos(theta-phi)) /(r**2 + 1 - 2*r*np.cos(theta-phi))
    return kernel


def h(phi):
    """Returns the right-hand side univariate function 
    in BIE"""
    return np.cos(2*phi)


def E(r, phi):
    """The exact solution of the Dirichlet BVP
    for Laplace equation"""
    return (r**2) * np.cos(2*phi)

In [3]:
phis = np.linspace(0, 2*np.pi, 300)
rs = np.linspace(1e-3, 0.5, 200, endpoint=False)
r_mesh, phis_mesh = np.meshgrid(rs, phis)
x_mesh, y_mesh = r_mesh * np.cos(phis_mesh), r_mesh * np.sin(phis_mesh)
E_mesh = np.zeros_like(r_mesh)
for i in range(len(rs)):
    for j in range(len(phis)):
            E_mesh[j][i] = E(rs[i], phis[j])

In [4]:
import shapely
from shapely.geometry.point import Point
p = Point(0, 0)
circle = p.buffer(0.5)
x, y = np.array(circle.exterior).T
z = np.zeros_like(x)

In [5]:
phi = np.linspace(0, 2*np.pi, 1200)
x1, y1, z1 = 0.5*np.cos(phi), 0.5*np.sin(phi), h(phi)

In [6]:
def taus_omegas_Spline32(a, b, x, w):
    
    taus_n = (b - a)/1 * x + a
    omegas_n = w*(b - a)/1
    
    omegas = np.append(omegas_n[:-1], omegas_n[::-1])
    taus = taus_n - taus_n.max() 
    taus = np.append(taus_n[:-1], -taus[::-1] + taus_n[-1])
    
    return taus, omegas
    

In [7]:
with open('TargetNodes_d=3_k=2_N=1.txt', 'r') as ff:
    x = np.loadtxt(ff)
with open('TargetWeights_d=3_k=2_N=1.txt', 'r') as ff:
    w = np.loadtxt(ff)
taus, omegas = taus_omegas_Spline32(0, 2*np.pi, x, w)

In [8]:
np.set_printoptions(precision=55)

In [9]:
taus

array([0.19162901076059377, 0.9438481647722337 , 2.0077252205171416 ,
       3.141592653589793  , 4.275460086662445  , 5.339337142407352  ,
       6.091556296418993  ])

## Spline 32

In [10]:
def test(N):
    a = 0
    b = 2 * np.pi

    print(N)    # number of macroblocks
    order = 7   # QP initial

    phis = np.pi/14 #np.linspace(0, 2*np.pi, 300)
    rs = 0.99 #np.linspace(1e-3, 1, 200, endpoint=False)

    #### define taus, omegas for integration with macroblocks

    with open('TargetNodes_d=3_k=2_N=1.txt', 'r') as ff:
        x = np.loadtxt(ff)
    with open('TargetWeights_d=3_k=2_N=1.txt', 'r') as ff:
        w = np.loadtxt(ff)
    
    interval = np.linspace(a, b, N+1)
    taus = []; omegas = []
    for a_, b_ in zip(interval[:-1], interval[1:]):
        t_, o_ = taus_omegas_Spline32(a_, b_, x, w)
        taus.append(t_)
        omegas.append(o_)
    taus = np.ravel(taus)
    omegas = np.ravel(omegas)
    #return taus, omegas
    #### block matrix and solution for sigmas

    Ko_blocks = []
    for taus_, omegas_ in zip(taus.reshape((N, order)),
                              omegas.reshape((N, order))):
        Ko_blocks.append(K_matrix(taus_, taus_) * omegas_)
    Ko = scipy.linalg.block_diag(*Ko_blocks)

    sigmas = np.dot(np.linalg.inv(np.identity(len(taus)) + Ko), 2*h(taus))

    #### Nystrom algorithm

    sigmas_ns = 2 * h(taus) - (K_matrix(taus, taus) * omegas) @ sigmas
    
    
    #### final integration

    rs_m, phis_m, thetas_m = np.meshgrid(rs, phis, taus, indexing='ij')
    ke_matrix = ke(rs_m, phis_m, thetas_m)

    integral = ((ke_matrix * sigmas_ns) * omegas).sum(axis=-1)

    rs_m, phis_m = np.meshgrid(rs, phis, indexing='ij')
    #x_mesh, y_mesh = rs_m * np.cos(phis_m), rs_m * np.sin(phis_m)
    u_mesh = 1/(2*np.pi) * integral

    E_mesh = E(rs_m, phis_m)
    
    #l2errors = np.sqrt(np.sum(np.power((E_mesh - u_mesh)*(phis[1]-phis[0])*(rs[1]-rs[0]),2)))

    return (abs(E_mesh - u_mesh)).max() #, l2errors
    #return taus, omegas

In [11]:
taus, omegas = test(2)

2


TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
taus

In [ ]:
plt.plot(taus, omegas)

In [ ]:
omegas

In [ ]:
print(round(errors[0], 2))

In [12]:
El = 8
errors = np.zeros(El)
me = np.zeros(El)
l2err = np.zeros(El)
for i in range(El):
    Ns = 2**i    #21 qp per element
    errors[i] = test(Ns)

1
2
4
8
16
32
64
128


In [13]:
ns = 7*2**np.arange(El) 
ms = ns
errs_rel = errors[:-1] / errors[1:]
errs_log = np.log2(errs_rel)


In [14]:
msg = '{:>4}\t{:>8}\t{:>12}'.format('m', 'Max Error', 'Log_2(Max Error)')

print(msg)
print('-'*(len(msg)+8))

for i in range(1, El): 
    if i == 1:
        print('{:4f}\t{:.2e}\t{:>12}'.format(ms[i], errors[i],  ''))
        
    else:
        print('{:.0f}\t{:.2e}\t{:.2e}'.format(ms[i], errors[i], errs_log[i-1]))

   m	Max Error	Log_2(Max Error)
---------------------------------------
14.000000	8.16e-01	            
28	2.07e+00	-1.34e+00
56	3.38e-01	2.62e+00
112	4.87e-01	-5.27e-01
224	2.45e-01	9.92e-01
448	1.93e-02	3.66e+00
896	1.17e-03	4.05e+00


In [ ]:
gauss2 = [9.4354934300106130e-01, 8.1666042231053249e-01,
       6.4336771182371377e-01, 3.5689045252101748e-01,
       5.7321554646218464e-02, 2.9676212035387950e-02,
       5.1145518910488130e-03, 5.9072467556320341e-05,
       7.2562886810345617e-09, 2.1127544158616729e-13]

In [ ]:
simpson = [1.56019771e+01, 7.32432908e+00, 3.20886295e+00, 1.19726659e+00,
       2.79304972e-01, 3.41503668e-02, 3.94664795e-02, 3.81571388e-03,
       2.26147370e-05, 7.66841812e-10]

In [ ]:
trapezoidal = [4.87661313e+01, 2.38930156e+01, 1.14665007e+01, 5.27327239e+00,
       2.21626804e+00, 7.63545739e-01, 1.65273660e-01, 1.17185553e-02,
       6.78534169e-05, 2.30149022e-09]

In [ ]:
errors[1:]

In [ ]:
plt.yscale('log')
plt.xscale('log')
h1, = plt.plot(ms[1:-2], errors[1:-2], 's-', color='red', label='Cubic Spline')
h2, = plt.plot(ms[1:-2], gauss2[:-2], 'v-', color='black', label='Polynomial Gauss, deg. 3')
h3, = plt.plot(ms[1:-2], simpson[:-2], 'x-', color='blue', label="Simpson's rule")
h4, = plt.plot(ms[1:-2], trapezoidal[:-2], 'o-', color='green', label='Trapezoidal rule')
plt.legend(handles=[h1, h2, h3, h4], loc=3, fancybox=True)
plt.xlabel("Quadrature points")
plt.ylabel("Error")
plt.savefig('images/ConvercenceCubic_099pi_cos2phi.png', dpi = 1000)

In [ ]:
errors